In [1]:
import findspark
findspark.init('/Users/kiranrudresha/Documents/spark/spark-2.2.0-bin-hadoop2.7')

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Recommend').getOrCreate()

In [4]:
movie_data = spark.read.csv('/Users/kiranrudresha/Documents/spark/pyspark/Python-and-Spark-for-Big-Data-master/Spark_for_Machine_Learning/'
                            'Recommender_Systems/movielens_ratings.csv',
                           inferSchema=True, header=True);

In [5]:
movie_data.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



In [6]:
train, test = movie_data.randomSplit([0.8,0.2])

In [7]:
train.count()

1180

In [8]:
test.count()

321

In [9]:
from pyspark.ml.recommendation import ALS

In [10]:
from pyspark.ml.evaluation import RegressionEvaluator

In [14]:
als = ALS(maxIter= 5 , regParam= 0.01,userCol='userId' , itemCol='movieId', ratingCol='rating')

In [12]:
type(als)

pyspark.ml.recommendation.ALS

In [15]:
als_model = als.fit(train)

In [16]:
predict = als_model.transform(test)

In [17]:
predict.show()

+-------+------+------+-----------+
|movieId|rating|userId| prediction|
+-------+------+------+-----------+
|     31|   1.0|    27|  1.8261023|
|     31|   3.0|     8|  0.9194347|
|     85|   1.0|    12|  2.7418673|
|     85|   1.0|     4|  1.4691871|
|     65|   2.0|     5|  1.9791893|
|     53|   3.0|    20|  1.7959032|
|     53|   5.0|     8|   3.125259|
|     78|   1.0|    12|  0.6738757|
|     78|   1.0|     1|  0.5593598|
|     78|   1.0|    13| 0.65550184|
|     78|   1.0|     2|-0.17809704|
|     34|   3.0|    25| -0.9369633|
|     81|   3.0|    26|  1.5488938|
|     81|   1.0|     1| -0.6375022|
|     81|   1.0|    16|-0.48860615|
|     81|   3.0|    18|  1.9608445|
|     28|   3.0|     1| -1.4511098|
|     76|   3.0|     3| 0.26806965|
|     76|   1.0|    19|-0.22894725|
|     76|   1.0|    25| -1.7183988|
+-------+------+------+-----------+
only showing top 20 rows



In [18]:
evaluator = RegressionEvaluator(metricName='rmse',labelCol='rating',predictionCol='prediction')

In [19]:
rmse = evaluator.evaluate(predict)

In [20]:
rmse
# very bad as the rating is from 1 to 5. 

1.762072573544678

In [23]:
new = test.filter(test['userId']==11).select(['movieId','userId'])

In [24]:
new.show()

+-------+------+
|movieId|userId|
+-------+------+
|      6|    11|
|     11|    11|
|     13|    11|
|     21|    11|
|     23|    11|
|     25|    11|
|     45|    11|
|     59|    11|
|     61|    11|
|     71|    11|
|     88|    11|
+-------+------+



In [26]:
new_predict = als_model.transform(new)

In [28]:
new_predict.orderBy('prediction', ascending = False)

DataFrame[movieId: int, userId: int, prediction: float]